# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv('./data/AviationData.csv', encoding='latin-1', low_memory=False)

**Datatypes**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

#### Summary Stats

In [4]:
df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [5]:
air_craft_se = df['Aircraft.Category'] # Series
print(f'NaN Values: {air_craft_se.isna().sum()}') # NaN values

print('\nCount per Category')
air_craft_se.value_counts() # count of non-NaN values

NaN Values: 56602

Count per Category


Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

**Reasonable Imputation**

In [6]:
air_craft_se.fillna('Airplane', inplace=True) # missing vals replaced by 'Airplane'

print('\nNew Count per Category (Imputation)')
air_craft_se.value_counts()


New Count per Category (Imputation)


Airplane             84219
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

**Airplane DataFrame**

In [7]:
airplane_df = df[air_craft_se == 'Airplane']
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84219 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                84219 non-null  object 
 1   Investigation.Type      84219 non-null  object 
 2   Accident.Number         84219 non-null  object 
 3   Event.Date              84219 non-null  object 
 4   Location                84169 non-null  object 
 5   Country                 83998 non-null  object 
 6   Latitude                30507 non-null  object 
 7   Longitude               30499 non-null  object 
 8   Airport.Code            48404 non-null  object 
 9   Airport.Name            50867 non-null  object 
 10  Injury.Severity         83289 non-null  object 
 11  Aircraft.damage         81201 non-null  object 
 12  Aircraft.Category       84219 non-null  object 
 13  Registration.Number     82949 non-null  object 
 14  Make                    84159 non-null

**Retain Only 'Professional Builds'**

In [8]:
airplane_df['Amateur.Built'].value_counts() # count no. of amateur builds

No     76008
Yes     8111
Name: Amateur.Built, dtype: int64

In [9]:
airplane_df = airplane_df[airplane_df['Amateur.Built'] == 'No'] # remove amateur builds
airplane_df['Amateur.Built'].value_counts()

No    76008
Name: Amateur.Built, dtype: int64

**Retain Events of the last 40 Years**

In [10]:
airplane_df = airplane_df.dropna(subset=['Publication.Date']) # drop NaNs
airplane_df['Report.Date'] = pd.to_datetime(airplane_df['Publication.Date']) # convert to pd datetime obj

airplane_df = airplane_df.sort_values(by=['Report.Date']) # sort by pd datetime asc
airplane_df = airplane_df[airplane_df['Report.Date'] > '1983'] # retain last 40 by filter

airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63340 entries, 217 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                63340 non-null  object        
 1   Investigation.Type      63340 non-null  object        
 2   Accident.Number         63340 non-null  object        
 3   Event.Date              63340 non-null  object        
 4   Location                63303 non-null  object        
 5   Country                 63181 non-null  object        
 6   Latitude                26131 non-null  object        
 7   Longitude               26126 non-null  object        
 8   Airport.Code            37108 non-null  object        
 9   Airport.Name            38562 non-null  object        
 10  Injury.Severity         62659 non-null  object        
 11  Aircraft.damage         60901 non-null  object        
 12  Aircraft.Category       63340 non-null  obje

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

**Filter for 'Injury' Columns**

In [11]:
filter_col = airplane_df.columns.str.contains('njur') # select injury cols to use as filter
inj_cols = airplane_df.columns[filter_col][1::] # select rows for injury columns

print('NaNs for Injury Columns')
airplane_df[inj_cols].isna().sum() # calc NaN per injury type

NaNs for Injury Columns


Total.Fatal.Injuries       9511
Total.Serious.Injuries    10411
Total.Minor.Injuries       9909
Total.Uninjured            4554
dtype: int64

In [12]:
airplane_df[inj_cols].isna().all(axis=1).sum()

140

**Remove NaNs on 'Injury' Columns**

In [13]:
airplane_df = airplane_df[~airplane_df[inj_cols].isna().all(axis=1)] # where true, remove injury NaNs from cols

print('\nRemaining \'true\' NaNs')
airplane_df[inj_cols].isna().sum()


Remaining 'true' NaNs


Total.Fatal.Injuries       9371
Total.Serious.Injuries    10271
Total.Minor.Injuries       9769
Total.Uninjured            4414
dtype: int64

**Replace Remaining NaNs with Zero**

In [14]:
airplane_df.loc[:,inj_cols] = airplane_df[inj_cols].fillna(0) # fill NaNs with zero
airplane_df[inj_cols].isna().sum() # NaNs are now absent

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

**Calculate Injury Rate**

In [15]:
airplane_df.loc[:,'N_passenger'] = airplane_df.loc[:,inj_cols].sum(axis=1) # total no. of passengers

airplane_df = airplane_df[airplane_df['N_passenger'] > 0] # only retain passenger records

passengers = airplane_df['N_passenger'] # passengers
serious_inj = airplane_df['Total.Serious.Injuries'] # serious injuries
fatal_inj = airplane_df['Total.Fatal.Injuries'] # fatal injuries

airplane_df['ser_inj_rate'] = (fatal_inj + serious_inj)/passengers # calculate serious injury rate

**View New Columns**

In [16]:
airplane_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report.Date,N_passenger,ser_inj_rate
217,20020917X02131,Accident,FTW82FQG14,1982-02-01,"PAMPA, TX",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Maneuvering,Probable Cause,01-02-1983,1983-01-02,2.0,1.0
218,20020917X02315,Accident,LAX82FVD09,1982-02-01,"SONOMA, CA",United States,NaN,NaN,SVA,SONOMA VALLEY,...,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,01-02-1983,1983-01-02,1.0,0.0
219,20020917X02170,Accident,LAX82DA059,1982-02-01,"LAS VEGAS, NV",United States,NaN,NaN,VGT,N. LAS VEGAS AIR TERMINAL,...,0.0,0.0,2.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02,2.0,0.0
220,20020917X02006,Accident,FTW82DA053,1982-02-01,"LAS CRUCES, NM",United States,NaN,NaN,LCR,CRAWFORD,...,0.0,0.0,3.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02,3.0,0.0
222,20020917X01922,Accident,DEN82DA028,1982-02-01,"ROCK SPRINGS, WY",United States,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,IMC,Cruise,Probable Cause,01-02-1983,1983-01-02,1.0,1.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

**Remove NaNs and unknowns**

In [17]:
airplane_df['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)

In [18]:
airplane_df['Aircraft.damage'].value_counts()

Substantial    46048
Destroyed      12494
Minor           1854
Unknown           41
Name: Aircraft.damage, dtype: int64

In [19]:
print('\nReplace unknowns with Nans')

airplane_df['Aircraft.damage'] = airplane_df['Aircraft.damage'].replace({'Unknown':np.nan}) # turn unkws to NaN
airplane_df.dropna(subset=['Aircraft.damage'], inplace=True) # drop NaNs from A.d col
airplane_df['Aircraft.damage'].value_counts() # confirm dropped NaNs


Replace unknowns with Nans


Substantial    46048
Destroyed      12494
Minor           1854
Name: Aircraft.damage, dtype: int64

**Boolean Masking for 'Destroyed'**

In [20]:
airplane_df['is_destroyed'] = (airplane_df['Aircraft.damage'] == 'Destroyed').astype(int)

In [21]:
airplane_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report.Date,N_passenger,ser_inj_rate,is_destroyed
217,20020917X02131,Accident,FTW82FQG14,1982-02-01,"PAMPA, TX",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,IMC,Maneuvering,Probable Cause,01-02-1983,1983-01-02,2.0,1.0,1
218,20020917X02315,Accident,LAX82FVD09,1982-02-01,"SONOMA, CA",United States,NaN,NaN,SVA,SONOMA VALLEY,...,0.0,1.0,VMC,Takeoff,Probable Cause,01-02-1983,1983-01-02,1.0,0.0,0
219,20020917X02170,Accident,LAX82DA059,1982-02-01,"LAS VEGAS, NV",United States,NaN,NaN,VGT,N. LAS VEGAS AIR TERMINAL,...,0.0,2.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02,2.0,0.0,0
220,20020917X02006,Accident,FTW82DA053,1982-02-01,"LAS CRUCES, NM",United States,NaN,NaN,LCR,CRAWFORD,...,0.0,3.0,VMC,Landing,Probable Cause,01-02-1983,1983-01-02,3.0,0.0,0
222,20020917X01922,Accident,DEN82DA028,1982-02-01,"ROCK SPRINGS, WY",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,IMC,Cruise,Probable Cause,01-02-1983,1983-01-02,1.0,1.0,0


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [22]:
airplane_df['Make'].value_counts()

Cessna                     17456
Piper                       9435
CESSNA                      4567
Beech                       3439
PIPER                       2668
                           ...  
Wsk Pzl-krosno                 1
STETSON NORMAN B               1
COMMONWEALTH                   1
FOLLAND AIRCRAFT CO LTD        1
SCOTT TERRY G                  1
Name: Make, Length: 1738, dtype: int64

**Consolidate**

In [23]:
airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["CESSNA AIRCRAFT CO", "CESSNA AIRCRAFT COMPANY", "CESSNA AIRCRAFT", "Cessna"], value="CESSNA")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["PIPER AIRCRAFT INC", "PIPER AIRCRAFT CORPORATION", "PIPER AIRCRAFT", "Piper"], value="PIPER")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["BEECHCRAFT", "HAWKER BEECHCRAFT", "HAWKER BEECHCRAFT CORP", "HAWKER BEECH", "Beech"], value="BEECH") 

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["THE BOEING COMPANY", "BOEING COMPANY", "BOEING STEARMAN", "Boeing"], value="BOEING")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["MOONEY AIRCRAFT CORP.", "MOONEY AIRPLANE CO INC", "MOONEY INTERNATIONAL", "Mooney"], value="MOONEY")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["GRUMMAN ACFT ENG COR-SCHWEIZER", "GRUMMAN AMERICAN AVN. CORP", "GRUMMAN AMERICAN AVN. CORP", "Grumman-schweizer", "Grumman"], value= "GRUMMAN")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AIRBUS INDUSTRIES", "Airbus Industrie", "Airbus"], value="AIRBUS")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["MAULE AIRCRAFT CORP", "Maule"], value="MAULE")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AERONCA AIRCRAFT CORPORATION", "AERONCA CHAMPION", "AERONCA CHAMP", "Aeronca", "Champion"], value="AERONCA")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AIR TRACTOR", "AIR TRACTOR INC", "Air Tractor"], value="AIR TRACTOR")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["CIRRUS", "CIRRUS DESIGN CORP", "CIRRUS DESIGN CORP.", "Cirrus"], value="CIRRUS")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["ERCOUPE", "ERCOUPE (ENG & RESEARCH CORP.)", "Ercoupe"], value="ERCOUPE")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AVIAT AIRCRAFT INC", "AVIAT", "Aviat"], value="AVIAT")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["ROCKWELL", "ROCKWELL INTERNATIONAL", "Rockwell"], value="ROCKWELL")

In [24]:
filtered_makes = airplane_df['Make'].value_counts()[airplane_df['Make'].value_counts() > 50]
filtered_makes[:25]

CESSNA               22063
PIPER                12136
BEECH                 4444
Bell                  1341
BOEING                1159
MOONEY                1135
GRUMMAN               1057
AERONCA                883
AIR TRACTOR            849
Bellanca               666
Robinson               592
Hughes                 524
MAULE                  502
Schweizer              427
CIRRUS                 346
ROCKWELL               320
Mcdonnell Douglas      314
De Havilland           293
Stinson                278
Aero Commander         273
Luscombe               259
North American         257
Taylorcraft            255
AVIAT                  209
Aerospatiale           206
Name: Make, dtype: int64

**Filter Based On Makes**

In [25]:
airplane_df = airplane_df[airplane_df['Make'].isin(filtered_makes.index)]

In [26]:
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54666 entries, 217 to 88784
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                54666 non-null  object        
 1   Investigation.Type      54666 non-null  object        
 2   Accident.Number         54666 non-null  object        
 3   Event.Date              54666 non-null  object        
 4   Location                54639 non-null  object        
 5   Country                 54544 non-null  object        
 6   Latitude                21632 non-null  object        
 7   Longitude               21629 non-null  object        
 8   Airport.Code            32597 non-null  object        
 9   Airport.Name            33979 non-null  object        
 10  Injury.Severity         54666 non-null  object        
 11  Aircraft.damage         54666 non-null  object        
 12  Aircraft.Category       54666 non-null  obje

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [27]:
airplane_df.dropna(subset=['Model'], inplace=True)

**Multi-Indexing (Make & Model)**

In [28]:
airplane_df['make_model'] = airplane_df['Make'] + '_' + airplane_df['Model'].str.upper()

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

**Clean 'Engine.Type'**

In [29]:
airplane_df['Engine.Type'].value_counts()

Reciprocating      46042
Turbo Prop          2307
Turbo Shaft         1551
Turbo Fan           1049
Unknown              871
Turbo Jet            266
Geared Turbofan        1
UNK                    1
Name: Engine.Type, dtype: int64

In [30]:
print('\nReplace \'Unknown\' & \'UNK\'')


Replace 'Unknown' & 'UNK'


In [31]:
airplane_df['Engine.Type'].replace({'Unknown': np.nan, 'UNK':np.nan}, inplace=True)

**Filter Out Single 'Engine' Values**

In [32]:
filtr_engine = airplane_df['Engine.Type'].value_counts()[airplane_df['Engine.Type'].value_counts() > 1] # filter out single engine value

airplane_df = airplane_df[airplane_df['Engine.Type'].isin(filtr_engine.index)] # implement filter in engine col

airplane_df['Engine.Type'].value_counts() # check work

Reciprocating    46042
Turbo Prop        2307
Turbo Shaft       1551
Turbo Fan         1049
Turbo Jet          266
Name: Engine.Type, dtype: int64

In [33]:
airplane_df['Number.of.Engines'].value_counts()

1.0    43821
2.0     6613
3.0      214
4.0      212
0.0        1
Name: Number.of.Engines, dtype: int64

**Removing Zero 'Number.of.Engines'**

In [34]:
airplane_df = airplane_df[airplane_df['Number.of.Engines'] > 0.0]

print('\nCheck Work')
airplane_df['Number.of.Engines'].value_counts()


Check Work


1.0    43821
2.0     6613
3.0      214
4.0      212
Name: Number.of.Engines, dtype: int64

**Clean 'Weather.Condition'**

In [35]:
print('\nUnique Values')
airplane_df['Weather.Condition'].unique()


Unique Values


array(['IMC', 'VMC', 'UNK', nan, 'Unk'], dtype=object)

**Replace 'Weather.Condition' Unknowns with NaNs**

In [36]:
airplane_df['Weather.Condition'].replace({'UNK': np.nan, 'Unk': np.nan}, inplace=True)

In [37]:
print('\n Check Work')
airplane_df['Weather.Condition'].value_counts()


 Check Work


VMC    46160
IMC     3916
Name: Weather.Condition, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [38]:
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50860 entries, 217 to 88767
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                50860 non-null  object        
 1   Investigation.Type      50860 non-null  object        
 2   Accident.Number         50860 non-null  object        
 3   Event.Date              50860 non-null  object        
 4   Location                50849 non-null  object        
 5   Country                 50741 non-null  object        
 6   Latitude                19570 non-null  object        
 7   Longitude               19567 non-null  object        
 8   Airport.Code            31212 non-null  object        
 9   Airport.Name            32613 non-null  object        
 10  Injury.Severity         50860 non-null  object        
 11  Aircraft.damage         50860 non-null  object        
 12  Aircraft.Category       50860 non-null  obje

**Drop Cols with Many NaNs**

In [39]:
airplane_df = airplane_df.drop(columns=['FAR.Description', 'Air.carrier', 'Schedule'])

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [40]:
airplane_df.to_csv('data/air_cleaned.csv', index=False)